## Conneaction and Importing Data

In [4]:
import bson

# Open and read the BSON file
with open('./Data/Gymnases.bson', 'rb') as file:
    data_gymnases = bson.decode_all(file.read())
with open('./Data/Sportifs.bson', 'rb') as file:
    data_sportifs = bson.decode_all(file.read())   



In [1]:
from pymongo import MongoClient
client=MongoClient('mongodb://localhost:27017/')
db=client["Gym"]

In [ ]:
Sportifs=db["sportifs"]
Gymnases=db["gymnases"]

In [8]:
db.sportifs.insert_many(data_sportifs)
db.gymnases.insert_many(data_gymnases)

In [2]:
for sportif in db.sportifs.find({}):
    print(sportif)

{'_id': '566eec5f662b388eba464203', 'IdSportif': 1, 'Nom': 'BOUTAHAR', 'Prenom': 'Abderahim', 'Sexe': 'm', 'Age': 30, 'Sports': {'Jouer': ['Volley ball', 'Tennis', 'Football'], 'Arbitrer': ['Basket ball', 'Volley ball', 'Hockey'], 'Entrainer': ['Basket ball', 'Volley ball', 'Hand ball', 'Hockey', 'Badmington']}}
{'_id': '566eec5f662b388eba464204', 'IdSportif': 2, 'Nom': 'KERVADEC', 'Prenom': 'Yann', 'Sexe': 'M', 'Age': 28, 'IdSportifConseiller': 1, 'Sports': {'Jouer': ['Basket ball', 'Volley ball', 'Ping pong', 'Football'], 'Arbitrer': ['Hockey', 'Football'], 'Entrainer': ['Basket ball', 'Volley ball', 'Hand ball', 'Tennis', 'Hockey', 'Badmington', 'Ping pong', 'Boxe']}}
{'_id': '566eec5f662b388eba464205', 'IdSportif': 3, 'Nom': 'HUE', 'Prenom': 'Pascale', 'Sexe': 'F', 'Age': 25, 'IdSportifConseiller': 1, 'Sports': {'Jouer': ['Volley ball', 'Ping pong'], 'Arbitrer': ['Volley ball', 'Badmington', 'Ping pong'], 'Entrainer': ['Basket ball', 'Volley ball', 'Hand ball', 'Badmington']}}
{'_i

In [3]:
for gymnase in db.gymnases.find({}):
    print(gymnase)

{'_id': '566eec69662b388eba464299', 'IdGymnase': 1, 'NomGymnase': 'PAUL ELUARD', 'Adresse': '2 rue des pépines', 'Ville': 'STAINS', 'Surface': 200, 'Seances': [{'IdSportifEntraineur': 149, 'Jour': 'Samedi', 'Horaire': 9.0, 'Duree': 60, 'Libelle': 'Basket ball'}, {'IdSportifEntraineur': 1, 'Jour': 'Lundi', 'Horaire': 9.0, 'Duree': 60, 'Libelle': 'Hand ball'}, {'IdSportifEntraineur': 1, 'Jour': 'Lundi', 'Horaire': 10.0, 'Duree': 60, 'Libelle': 'Hand ball'}, {'IdSportifEntraineur': 1, 'Jour': 'Lundi', 'Horaire': 11.3, 'Duree': 60, 'Libelle': 'Hand ball'}, {'IdSportifEntraineur': 1, 'Jour': 'Lundi', 'Horaire': 14.0, 'Duree': 90, 'Libelle': 'Hand ball'}, {'IdSportifEntraineur': 1, 'Jour': 'lundi', 'Horaire': 17.3, 'Duree': 120, 'Libelle': 'Hand ball'}, {'IdSportifEntraineur': 1, 'Jour': 'Lundi', 'Horaire': 19.3, 'Duree': 120, 'Libelle': 'Hand ball'}, {'IdSportifEntraineur': 2, 'Jour': 'Dimanche', 'Horaire': 17.3, 'Duree': 120, 'Libelle': 'Hand ball'}, {'IdSportifEntraineur': 2, 'Jour': 'Dim

## Understanding Aggregation

### Aggregation $Group

In [4]:
## 
pipeline = [
    {
        "$group": {
            "_id": "$Age",  # Group by Age
            "totalSportifs": {"$sum": 1}  # Count sportifs for each age
        }
    }
]

results = db.sportifs.aggregate(pipeline)
for sportif in results:
    print(sportif)

{'_id': 32, 'totalSportifs': 4}
{'_id': 27, 'totalSportifs': 3}
{'_id': 28, 'totalSportifs': 4}
{'_id': 22, 'totalSportifs': 13}
{'_id': 25, 'totalSportifs': 18}
{'_id': 26, 'totalSportifs': 10}
{'_id': 40, 'totalSportifs': 2}
{'_id': 24, 'totalSportifs': 13}
{'_id': 36, 'totalSportifs': 1}
{'_id': 39, 'totalSportifs': 1}
{'_id': 23, 'totalSportifs': 16}
{'_id': 30, 'totalSportifs': 65}


In [6]:
pipeline = [
    {
        "$group": {
            "_id": "$Ville",  # Group by the "Ville" (City)
            "averageSurface": {"$avg": "$Surface"}  # Calculate average gym surface
        }
    }
]

results = db.gymnases.aggregate(pipeline)
for gymnase in results:
    print(gymnase)


{'_id': 'STAINS', 'averageSurface': 333.3333333333333}
{'_id': 'MONTMORENCY', 'averageSurface': 470.0}
{'_id': 'SARCELLES', 'averageSurface': 548.0}
{'_id': 'GARGES', 'averageSurface': 400.0}
{'_id': 'SAINT DENIS', 'averageSurface': 490.0}
{'_id': 'PIERREFITTE', 'averageSurface': 382.0}
{'_id': 'VILLETANEUSE', 'averageSurface': 523.3333333333334}


In [9]:
pipeline = [
    {
        "$unwind": "$Seances"  # Unwind the Seances array to process each session
    },
    {
        "$group": {
            "_id": "$Seances.Libelle",  # Group by sport (Libelle field)
            "totalSessions": {"$sum": 1}  # Count how many sessions per sport
        }
    }
]
results = db.gymnases.aggregate(pipeline)
for gymnase in results:
    print(gymnase)

{'_id': 'Hand ball', 'totalSessions': 20}
{'_id': 'Basket ball', 'totalSessions': 7}
{'_id': 'Hockey', 'totalSessions': 27}
{'_id': 'Volley ball', 'totalSessions': 28}


### Aggregation $limit

#### 
The $limit stage in MongoDB's aggregation framework is used to restrict the number of documents returned by an aggregation pipeline.

In [27]:
pipeline = [
    {
        "$unwind": "$Seances"  # Unwind the Seances array to process each session separately
    },
    {
        "$group": {
            "_id": "$Ville",  # Group by the city (Ville)
            "totalSeances": {"$sum": 1}  # Count total number of sessions
        }
    }
]

results = db.gymnases.aggregate(pipeline)
for gymnase in results:
    print(gymnase)


{'_id': 'PIERREFITTE', 'totalSeances': 8}
{'_id': 'SAINT DENIS', 'totalSeances': 3}
{'_id': 'VILLETANEUSE', 'totalSeances': 4}
{'_id': 'GARGES', 'totalSeances': 8}
{'_id': 'STAINS', 'totalSeances': 37}
{'_id': 'MONTMORENCY', 'totalSeances': 10}
{'_id': 'SARCELLES', 'totalSeances': 12}


In [34]:
pipeline = [
    {
        "$unwind": "$Seances"  # Unwind the Seances array to access individual sessions
    },
    {
        "$match": {
            "Seances.Libelle": "Hand ball",  # Filter for "Hand ball" sessions
            "Seances.Jour":"Samedi" #  Filter for "Samedi" sessions
        }
    },
    
    {
        "$limit": 2  # Limit the result to 2 gymnases
    }
    
    
]

results = db.gymnases.aggregate(pipeline)
for gymnase in results:
    print(gymnase)

{'_id': '566eec69662b388eba464299', 'IdGymnase': 1, 'NomGymnase': 'PAUL ELUARD', 'Adresse': '2 rue des pépines', 'Ville': 'STAINS', 'Surface': 200, 'Seances': {'IdSportifEntraineur': 2, 'Jour': 'Samedi', 'Horaire': 15.3, 'Duree': 60, 'Libelle': 'Hand ball'}}
{'_id': '566eec69662b388eba464299', 'IdGymnase': 1, 'NomGymnase': 'PAUL ELUARD', 'Adresse': '2 rue des pépines', 'Ville': 'STAINS', 'Surface': 200, 'Seances': {'IdSportifEntraineur': 2, 'Jour': 'Samedi', 'Horaire': 16.3, 'Duree': 60, 'Libelle': 'Hand ball'}}


### Aggregation $match

##### The $match stage in MongoDB's aggregation framework is used to filter documents based on specified criteria. It works similarly to the find() method but is part of the aggregation pipeline. You typically use $match at the beginning or middle of a pipeline to limit the documents that are processed by subsequent stages

In [35]:
pipeline = [
    {
        "$match": {
            "Surface": {"$gt": 400}  # Filter for gymnases with a surface area > 400
        }
    }
]

results = db.gymnases.aggregate(pipeline)
for gymnase in results:
    print(gymnase)


{'_id': '566eec69662b388eba46429a', 'IdGymnase': 2, 'NomGymnase': 'ARAGON', 'Adresse': 'Place du Chartres', 'Ville': 'MONTMORENCY', 'Surface': 450, 'Seances': [{'IdSportifEntraineur': 57, 'Jour': 'dimanche', 'Horaire': 17.0, 'Duree': 60, 'Libelle': 'Volley ball'}]}
{'_id': '566eec69662b388eba46429c', 'IdGymnase': 4, 'NomGymnase': 'PAUL ELUARD', 'Adresse': 'Allée J.B. Lulli', 'Ville': 'SARCELLES', 'Surface': 500, 'Seances': [{'IdSportifEntraineur': 149, 'Jour': 'Vendredi', 'Horaire': 10.0, 'Duree': 30, 'Libelle': 'Basket ball'}, {'IdSportifEntraineur': 6, 'Jour': 'mercredi', 'Horaire': 19.0, 'Duree': 60, 'Libelle': 'Hockey'}]}
{'_id': '566eec69662b388eba46429d', 'IdGymnase': 5, 'NomGymnase': 'BRASSENS', 'Adresse': '153 square Loliot', 'Ville': 'SARCELLES', 'Surface': 620, 'Seances': [{'IdSportifEntraineur': 57, 'Jour': 'lundi', 'Horaire': 16.3, 'Duree': 90, 'Libelle': 'Volley ball'}, {'IdSportifEntraineur': 6, 'Jour': 'jeudi', 'Horaire': 19.0, 'Duree': 60, 'Libelle': 'Hockey'}]}
{'_id':

In [36]:
pipeline = [
    {
        "$match": {
            "Surface": {"$gt": 400}  # Only consider gymnases with surface > 400
        }
    },
    {
        "$group": {
            "_id": "$Ville",  # Group by city
            "totalGymnases": {"$sum": 1}  # Count the number of gymnases in each city
        }
    }
]

results = db.gymnases.aggregate(pipeline)
for result in results:
    print(result)


{'_id': 'MONTMORENCY', 'totalGymnases': 5}
{'_id': 'SARCELLES', 'totalGymnases': 4}
{'_id': 'PIERREFITTE', 'totalGymnases': 1}
{'_id': 'SAINT DENIS', 'totalGymnases': 3}
{'_id': 'VILLETANEUSE', 'totalGymnases': 2}


In [37]:
pipeline = [
    {
        "$match": {
            "$and": [
                {"Ville": "SARCELLES"},
                {"Surface": {"$gt": 500}}  # Only gymnases with a surface > 500
            ]
        }
    }
]

results = db.gymnases.aggregate(pipeline)
for gymnase in results:
    print(gymnase)


{'_id': '566eec69662b388eba46429d', 'IdGymnase': 5, 'NomGymnase': 'BRASSENS', 'Adresse': '153 square Loliot', 'Ville': 'SARCELLES', 'Surface': 620, 'Seances': [{'IdSportifEntraineur': 57, 'Jour': 'lundi', 'Horaire': 16.3, 'Duree': 90, 'Libelle': 'Volley ball'}, {'IdSportifEntraineur': 6, 'Jour': 'jeudi', 'Horaire': 19.0, 'Duree': 60, 'Libelle': 'Hockey'}]}
{'_id': '566eec69662b388eba4642a1', 'IdGymnase': 9, 'NomGymnase': 'CAMUS', 'Adresse': '3 esplanade des quatrans', 'Ville': 'SARCELLES', 'Surface': 620, 'Seances': [{'IdSportifEntraineur': 6, 'Jour': 'samedi', 'Horaire': 19.0, 'Duree': 60, 'Libelle': 'Hockey'}]}
{'_id': '566eec69662b388eba4642a8', 'IdGymnase': 16, 'NomGymnase': 'SAMOURAI', 'Adresse': '4 Allée des pendules', 'Ville': 'SARCELLES', 'Surface': 600, 'Seances': [{'IdSportifEntraineur': 57, 'Jour': 'lundi', 'Horaire': 16.3, 'Duree': 90, 'Libelle': 'Volley ball'}, {'IdSportifEntraineur': 60, 'Jour': 'Lundi', 'Horaire': 17.0, 'Duree': 60, 'Libelle': 'Volley ball'}, {'IdSportif

In [38]:
pipeline = [
    {
        "$match": {
            "$or": [
                {"Ville": "SARCELLES"},
                {"Surface": {"$gt": 500}}  # Only gymnases with a surface > 500
            ]
        }
    }
]

results = db.gymnases.aggregate(pipeline)
for gymnase in results:
    print(gymnase)


{'_id': '566eec69662b388eba46429c', 'IdGymnase': 4, 'NomGymnase': 'PAUL ELUARD', 'Adresse': 'Allée J.B. Lulli', 'Ville': 'SARCELLES', 'Surface': 500, 'Seances': [{'IdSportifEntraineur': 149, 'Jour': 'Vendredi', 'Horaire': 10.0, 'Duree': 30, 'Libelle': 'Basket ball'}, {'IdSportifEntraineur': 6, 'Jour': 'mercredi', 'Horaire': 19.0, 'Duree': 60, 'Libelle': 'Hockey'}]}
{'_id': '566eec69662b388eba46429d', 'IdGymnase': 5, 'NomGymnase': 'BRASSENS', 'Adresse': '153 square Loliot', 'Ville': 'SARCELLES', 'Surface': 620, 'Seances': [{'IdSportifEntraineur': 57, 'Jour': 'lundi', 'Horaire': 16.3, 'Duree': 90, 'Libelle': 'Volley ball'}, {'IdSportifEntraineur': 6, 'Jour': 'jeudi', 'Horaire': 19.0, 'Duree': 60, 'Libelle': 'Hockey'}]}
{'_id': '566eec69662b388eba4642a1', 'IdGymnase': 9, 'NomGymnase': 'CAMUS', 'Adresse': '3 esplanade des quatrans', 'Ville': 'SARCELLES', 'Surface': 620, 'Seances': [{'IdSportifEntraineur': 6, 'Jour': 'samedi', 'Horaire': 19.0, 'Duree': 60, 'Libelle': 'Hockey'}]}
{'_id': '56

### Aggregation $project

###### The $project stage in MongoDB's aggregation framework is used to include, exclude, and transform fields in the documents that pass through the pipeline. It is similar to the projection argument in a find() query, but more powerful because it allows for additional operations like renaming fields, adding computed fields, or modifying the structure of the output.

In [39]:
pipeline = [
    {
        "$project": {
            "NomGymnase": 1,  # Include gym name
            "Ville": 1,  # Include city
            "_id": 0  # Exclude the _id field
        }
    }
]

results = db.gymnases.aggregate(pipeline)
for gymnase in results:
    print(gymnase)


{'NomGymnase': 'PAUL ELUARD', 'Ville': 'STAINS'}
{'NomGymnase': 'ARAGON', 'Ville': 'MONTMORENCY'}
{'NomGymnase': 'SAINT EXUPERY', 'Ville': 'PIERREFITTE'}
{'NomGymnase': 'PAUL ELUARD', 'Ville': 'SARCELLES'}
{'NomGymnase': 'BRASSENS', 'Ville': 'SARCELLES'}
{'NomGymnase': 'VERLAINE', 'Ville': 'STAINS'}
{'NomGymnase': 'JULES FERRY', 'Ville': 'PIERREFITTE'}
{'NomGymnase': 'PREVERT', 'Ville': 'MONTMORENCY'}
{'NomGymnase': 'CAMUS', 'Ville': 'SARCELLES'}
{'NomGymnase': 'RIMBAUD', 'Ville': 'STAINS'}
{'NomGymnase': 'LAMARTINE', 'Ville': 'PIERREFITTE'}
{'NomGymnase': 'MOZART', 'Ville': 'MONTMORENCY'}
{'NomGymnase': 'RAVEL', 'Ville': 'STAINS'}
{'NomGymnase': 'CHOPIN', 'Ville': 'MONTMORENCY'}
{'NomGymnase': 'BREL', 'Ville': 'PIERREFITTE'}
{'NomGymnase': 'SAMOURAI', 'Ville': 'SARCELLES'}
{'NomGymnase': 'GARCIA LORCA', 'Ville': 'STAINS'}
{'NomGymnase': 'PABLO NERUDA', 'Ville': 'PIERREFITTE'}
{'NomGymnase': 'COCTEAU', 'Ville': 'MONTMORENCY'}
{'NomGymnase': 'LUMIERES', 'Ville': 'SARCELLES'}
{'NomGymnas

In [40]:
pipeline = [
    {
        "$project": {
            "NomGymnase": 1,  # Include gym name
            "Ville": 1,  # Include city
            "LargeSurface": { "$gt": ["$Surface", 500] },  # Add a computed field
            "_id": 0  # Exclude the _id field
        }
    }
]

results = db.gymnases.aggregate(pipeline)
for gymnase in results:
    print(gymnase)


{'NomGymnase': 'PAUL ELUARD', 'Ville': 'STAINS', 'LargeSurface': False}
{'NomGymnase': 'ARAGON', 'Ville': 'MONTMORENCY', 'LargeSurface': False}
{'NomGymnase': 'SAINT EXUPERY', 'Ville': 'PIERREFITTE', 'LargeSurface': False}
{'NomGymnase': 'PAUL ELUARD', 'Ville': 'SARCELLES', 'LargeSurface': False}
{'NomGymnase': 'BRASSENS', 'Ville': 'SARCELLES', 'LargeSurface': True}
{'NomGymnase': 'VERLAINE', 'Ville': 'STAINS', 'LargeSurface': False}
{'NomGymnase': 'JULES FERRY', 'Ville': 'PIERREFITTE', 'LargeSurface': False}
{'NomGymnase': 'PREVERT', 'Ville': 'MONTMORENCY', 'LargeSurface': False}
{'NomGymnase': 'CAMUS', 'Ville': 'SARCELLES', 'LargeSurface': True}
{'NomGymnase': 'RIMBAUD', 'Ville': 'STAINS', 'LargeSurface': False}
{'NomGymnase': 'LAMARTINE', 'Ville': 'PIERREFITTE', 'LargeSurface': False}
{'NomGymnase': 'MOZART', 'Ville': 'MONTMORENCY', 'LargeSurface': False}
{'NomGymnase': 'RAVEL', 'Ville': 'STAINS', 'LargeSurface': False}
{'NomGymnase': 'CHOPIN', 'Ville': 'MONTMORENCY', 'LargeSurface'

In [41]:
pipeline = [
    {
        "$project": {
            "IdSportif": 1,  # Include the sportif ID
            "FullName": { "$concat": ["$Prenom", " ", "$Nom"] },  # Concatenate first name and last name
            "_id": 0  # Exclude the _id field
        }
    }
]

results = db.sportifs.aggregate(pipeline)
for sportif in results:
    print(sportif)


{'IdSportif': 1, 'FullName': 'Abderahim BOUTAHAR'}
{'IdSportif': 2, 'FullName': 'Yann KERVADEC'}
{'IdSportif': 3, 'FullName': 'Pascale HUE'}
{'IdSportif': 4, 'FullName': 'Jean-michel DORLEANS'}
{'IdSportif': 5, 'FullName': 'Sylvie COMES'}
{'IdSportif': 6, 'FullName': 'Sophie RETALDI'}
{'IdSportif': 7, 'FullName': 'Diego GOMEZ'}
{'IdSportif': 9, 'FullName': 'Gaêlle ANTUNES'}
{'IdSportif': 10, 'FullName': 'Arnaud BLANDET'}
{'IdSportif': 11, 'FullName': 'Christophe BONNET'}
{'IdSportif': 12, 'FullName': 'Benoît BORREL'}
{'IdSportif': 13, 'FullName': 'Julien BOYON'}
{'IdSportif': 14, 'FullName': 'Stéphanie CAILLIOT'}
{'IdSportif': 15, 'FullName': 'Cédric CHAMPENOIS'}
{'IdSportif': 16, 'FullName': 'Michaël COLAS'}
{'IdSportif': 17, 'FullName': 'Michaël COLOMB'}
{'IdSportif': 18, 'FullName': 'Leila HOSNI'}
{'IdSportif': 19, 'FullName': 'Laurent TESTEMONT'}
{'IdSportif': 20, 'FullName': 'Thomas TIENER'}
{'IdSportif': 21, 'FullName': 'Denis LE BOUCHER'}
{'IdSportif': 22, 'FullName': 'Pierre LE

### Aggregation $Sort

In [43]:
pipeline = [
    {
        "$sort": {
            "Surface": -1  # Sort by Surface in descending order
        }
    }
]

results = db.gymnases.aggregate(pipeline)
for gymnase in results:
    print(gymnase)


{'_id': '566eec69662b388eba46429d', 'IdGymnase': 5, 'NomGymnase': 'BRASSENS', 'Adresse': '153 square Loliot', 'Ville': 'SARCELLES', 'Surface': 620, 'Seances': [{'IdSportifEntraineur': 57, 'Jour': 'lundi', 'Horaire': 16.3, 'Duree': 90, 'Libelle': 'Volley ball'}, {'IdSportifEntraineur': 6, 'Jour': 'jeudi', 'Horaire': 19.0, 'Duree': 60, 'Libelle': 'Hockey'}]}
{'_id': '566eec69662b388eba4642a1', 'IdGymnase': 9, 'NomGymnase': 'CAMUS', 'Adresse': '3 esplanade des quatrans', 'Ville': 'SARCELLES', 'Surface': 620, 'Seances': [{'IdSportifEntraineur': 6, 'Jour': 'samedi', 'Horaire': 19.0, 'Duree': 60, 'Libelle': 'Hockey'}]}
{'_id': '566eec69662b388eba4642b1', 'IdGymnase': 25, 'NomGymnase': 'DOLTO', 'Adresse': '3 square Plaisir', 'Ville': 'VILLETANEUSE', 'Surface': 620, 'Seances': [{'IdSportifEntraineur': 149, 'Jour': 'Dimanche', 'Horaire': 18.0, 'Duree': 60, 'Libelle': 'Basket ball'}]}
{'_id': '566eec69662b388eba4642a8', 'IdGymnase': 16, 'NomGymnase': 'SAMOURAI', 'Adresse': '4 Allée des pendules'

In [45]:
pipeline = [
    {
        "$sort": {
            "Age": 1,  # Sort by Age in ascending order
            "Nom": 1   # Sort by Name in alphabetical order for same Age
        }
    },
    {
        "$limit": 10  # Limit the results to the top 10
    }
]

results = db.sportifs.aggregate(pipeline)
for sportif in results:
    print(sportif)


{'_id': '566eec5f662b388eba46420e', 'IdSportif': 13, 'Nom': 'BOYON', 'Prenom': 'Julien', 'Sexe': 'M', 'Age': 22, 'IdSportifConseiller': 4, 'Sports': {'Jouer': ['Volley ball', 'Badmington', 'Football']}}
{'_id': '566eec5f662b388eba464212', 'IdSportif': 17, 'Nom': 'COLOMB', 'Prenom': 'Michaël', 'Sexe': 'M', 'Age': 22, 'IdSportifConseiller': 7, 'Sports': {'Jouer': ['Volley ball', 'Badmington', 'Football']}}
{'_id': '566eec5f662b388eba464207', 'IdSportif': 5, 'Nom': 'COMES', 'Prenom': 'Sylvie', 'Sexe': 'F', 'Age': 22, 'IdSportifConseiller': 1, 'Sports': {'Jouer': ['Basket ball', 'Volley ball', 'Badmington', 'Ping pong'], 'Arbitrer': 'Ping pong'}}
{'_id': '566eec60662b388eba464235', 'IdSportif': 52, 'Nom': 'DANDOIS', 'Prenom': 'Régis', 'Sexe': 'M', 'Age': 22, 'IdSportifConseiller': 5, 'Sports': {'Jouer': ['Basket ball', 'Badmington', 'Ping pong', 'Football']}}
{'_id': '566eec60662b388eba46422a', 'IdSportif': 41, 'Nom': 'DELOVINA', 'Prenom': 'Elina', 'Sexe': 'F', 'Age': 22, 'IdSportifConseil

In [48]:
pipeline = [
    {
        "$unwind": "$Seances"  # Unwind Seances array to access individual sessions
    },
    {
        "$sort": {
            "Seances.Horaire": 1  # Sort sessions by Horaire (time) in ascending order
        }
    },
    {
        "$group": {
            "_id": "$NomGymnase",  # Group by gymnase
            "Seances": { "$push": "$Seances" }  # Rebuild the Seances array sorted by time
        }
    }
]

results = db.gymnases.aggregate(pipeline)
for gymnase in results:
    print(gymnase)


{'_id': 'BARBARA', 'Seances': [{'IdSportifEntraineur': 57, 'Jour': 'mardi', 'Horaire': 10.0, 'Duree': 30, 'Libelle': 'Volley ball'}]}
{'_id': 'BREL', 'Seances': [{'IdSportifEntraineur': 57, 'Jour': 'lundi', 'Horaire': 16.3, 'Duree': 90, 'Libelle': 'Volley ball'}]}
{'_id': 'RAVEL', 'Seances': [{'IdSportifEntraineur': 7, 'Jour': 'lundi', 'Horaire': 17.0, 'Duree': 90, 'Libelle': 'Hockey'}, {'IdSportifEntraineur': 60, 'Jour': 'dimanche', 'Horaire': 19.0, 'Duree': 60, 'Libelle': 'Volley ball'}, {'IdSportifEntraineur': 6, 'Jour': 'mercredi', 'Horaire': 20.0, 'Duree': 60, 'Libelle': 'Hockey'}]}
{'_id': 'CAMUS', 'Seances': [{'IdSportifEntraineur': 6, 'Jour': 'samedi', 'Horaire': 19.0, 'Duree': 60, 'Libelle': 'Hockey'}]}
{'_id': 'PABLO NERUDA', 'Seances': [{'IdSportifEntraineur': 7, 'Jour': 'mercredi', 'Horaire': 14.0, 'Duree': 120, 'Libelle': 'Hockey'}, {'IdSportifEntraineur': 7, 'Jour': 'mercredi', 'Horaire': 16.0, 'Duree': 90, 'Libelle': 'Hockey'}, {'IdSportifEntraineur': 57, 'Jour': 'lundi'

### Aggregation $count

###### In MongoDB, you can use the $count stage in the aggregation pipeline to count the number of documents that match certain criteria. It works similarly to countDocuments() but can be part of an aggregation pipeline, allowing you to filter, group, and perform other transformations before counting.



In [49]:
pipeline = [
    {
        "$count": "totalGymnases"  # Count all documents in the collection
    }
]

results = db.gymnases.aggregate(pipeline)
for result in results:
    print(result)


{'totalGymnases': 28}


In [50]:
pipeline = [
    {
        "$match": {
            "Surface": {"$gt": 500}  # Filter gymnases with surface > 500
        }
    },
    {
        "$count": "totalLargeGymnases"  # Count the remaining documents
    }
]

results = db.gymnases.aggregate(pipeline)
for result in results:
    print(result)


{'totalLargeGymnases': 6}


In [62]:
pipeline = [
    {
        "$unwind": "$Seances"  # Unwind the Seances array to process each session individually
    },
    {
        "$match": {
            "Seances.Libelle": "Hand ball"  # Filter for handball sessions
        }
    },
    {
        "$group": {
            "_id": "$NomGymnase"  # Group by gymnase name to avoid counting duplicate sessions
        }
    },
    {
        "$count": "totalHandballGymnases"  # Count unique gymnases offering handball
    }
]

results = db.gymnases.aggregate(pipeline)
for result in results:
    print(result)


{'totalHandballGymnases': 2}


### Aggregation $lookup

###### The $lookup stage in MongoDB’s aggregation framework is used to perform a join between two collections. It works similarly to SQL’s JOIN operation, allowing you to combine documents from different collections based on a related field.



In [63]:
pipeline = [
    {
        "$lookup": {
            "from": "gymnases",  # Join with the "gymnases" collection
            "localField": "Ville",  # Match "Ville" in "sportifs" collection
            "foreignField": "Ville",  # Match "Ville" in "gymnases" collection
            "as": "gymnasesInSameCity"  # Store matched gymnases in this array
        }
    }
]

results = db.sportifs.aggregate(pipeline)
for sportif in results:
    print(sportif)


{'_id': '566eec5f662b388eba464203', 'IdSportif': 1, 'Nom': 'BOUTAHAR', 'Prenom': 'Abderahim', 'Sexe': 'm', 'Age': 30, 'Sports': {'Jouer': ['Volley ball', 'Tennis', 'Football'], 'Arbitrer': ['Basket ball', 'Volley ball', 'Hockey'], 'Entrainer': ['Basket ball', 'Volley ball', 'Hand ball', 'Hockey', 'Badmington']}, 'gymnasesInSameCity': []}
{'_id': '566eec5f662b388eba464204', 'IdSportif': 2, 'Nom': 'KERVADEC', 'Prenom': 'Yann', 'Sexe': 'M', 'Age': 28, 'IdSportifConseiller': 1, 'Sports': {'Jouer': ['Basket ball', 'Volley ball', 'Ping pong', 'Football'], 'Arbitrer': ['Hockey', 'Football'], 'Entrainer': ['Basket ball', 'Volley ball', 'Hand ball', 'Tennis', 'Hockey', 'Badmington', 'Ping pong', 'Boxe']}, 'gymnasesInSameCity': []}
{'_id': '566eec5f662b388eba464205', 'IdSportif': 3, 'Nom': 'HUE', 'Prenom': 'Pascale', 'Sexe': 'F', 'Age': 25, 'IdSportifConseiller': 1, 'Sports': {'Jouer': ['Volley ball', 'Ping pong'], 'Arbitrer': ['Volley ball', 'Badmington', 'Ping pong'], 'Entrainer': ['Basket bal

In [64]:
pipeline = [
    {
        "$lookup": {
            "from": "sessions",  # Join with the "sessions" collection
            "localField": "IdSportif",  # Use the IdSportif field in the sportifs collection
            "foreignField": "IdSportif",  # Match with IdSportif in the sessions collection
            "as": "trainingSessions"  # Store matched sessions in an array
        }
    }
]

results = db.sportifs.aggregate(pipeline)
for sportif in results:
    print(sportif)


{'_id': '566eec5f662b388eba464203', 'IdSportif': 1, 'Nom': 'BOUTAHAR', 'Prenom': 'Abderahim', 'Sexe': 'm', 'Age': 30, 'Sports': {'Jouer': ['Volley ball', 'Tennis', 'Football'], 'Arbitrer': ['Basket ball', 'Volley ball', 'Hockey'], 'Entrainer': ['Basket ball', 'Volley ball', 'Hand ball', 'Hockey', 'Badmington']}, 'trainingSessions': []}
{'_id': '566eec5f662b388eba464204', 'IdSportif': 2, 'Nom': 'KERVADEC', 'Prenom': 'Yann', 'Sexe': 'M', 'Age': 28, 'IdSportifConseiller': 1, 'Sports': {'Jouer': ['Basket ball', 'Volley ball', 'Ping pong', 'Football'], 'Arbitrer': ['Hockey', 'Football'], 'Entrainer': ['Basket ball', 'Volley ball', 'Hand ball', 'Tennis', 'Hockey', 'Badmington', 'Ping pong', 'Boxe']}, 'trainingSessions': []}
{'_id': '566eec5f662b388eba464205', 'IdSportif': 3, 'Nom': 'HUE', 'Prenom': 'Pascale', 'Sexe': 'F', 'Age': 25, 'IdSportifConseiller': 1, 'Sports': {'Jouer': ['Volley ball', 'Ping pong'], 'Arbitrer': ['Volley ball', 'Badmington', 'Ping pong'], 'Entrainer': ['Basket ball', 